# roBERTo Classifier

This notebook inspired in BERT/roBERTa models andd attaches an output layer that works as multiclassification. As well, this model is trained with a spanish corpus.

In [ ]:
!pip install -r requirements.txt

In [1]:
from datasets import load_dataset

dataset = load_dataset("catalonia_independence")

Downloading: 5.99kB [00:00, 60.2kB/s]                   
Downloading: 7.64kB [00:00, 118kB/s]                    
No config specified, defaulting to: catalonia_independence/catalan


Downloading: 100%|██████████| 995k/995k [00:06<00:00, 157kB/s]


Dataset catalonia_independence downloaded and prepared to /Users/ernestomancebo/.cache/huggingface/datasets/catalonia_independence/catalan/1.1.0/e03c46e26a08e770675760d05c1a923879381d46d19318682e8e2ddfcfc6ef19. Subsequent calls will reuse this data.


In [3]:
train = dataset['train']

In [9]:
train[:15]

{'id_str': ['11028517837209518e+18',
  '1098892781681082368',
  '10990323264895959e+18',
  '110251884676898e+18',
  '1093081744193081344',
  '1097490620204101632',
  '10363613732388004e+18',
  '10383490390524355e+18',
  '10994375828558725e+18',
  '1045062904335265795',
  '1102943734428246e+18',
  '1046425772653727746',
  '1041394524281602048',
  '11030354070868746e+18',
  '10932692115295725e+18'],
 'TWEET': ['En @fgarrobo ha fet més per l’independentisme i la unitat popular a Nou Barris que tots aquests que tant el critiquen als seus barris i pobles. Amunt Francisco!✊🏾',
  "Divendres 22F acte Cloenda campanya de les candidatures q es presenten a la jornada de votació @PrimariesCat d'aquest dissabte 23F!! Escoltarem les candidates i candidats q sí creuen en una nova manera de fer política!!   #CanviemElGuió #AUTOconvocats #21FVagaGeneral #judicifarsa https://t.co/mtKiZLQUlv",
  "Puigdemont porta a la Defensora del Poble europea la prohibició de l'acte a l'Eurocambra https://t.co/fera2Vd

In [11]:
from datasets import load_dataset

muchocine_dataset = load_dataset("muchocine")

Downloading: 3.86kB [00:00, 673kB/s]                    
Downloading: 1.10kB [00:00, 343kB/s]                  
Using custom data configuration default


Downloading: 100%|██████████| 55.6M/55.6M [02:02<00:00, 454kB/s] 


Dataset muchocine downloaded and prepared to /Users/ernestomancebo/.cache/huggingface/datasets/muchocine/default/1.1.1/3ed5582584cd84ef722606a3d725ef18fd4647d63195fef05c47683e5a056ccd. Subsequent calls will reuse this data.


In [12]:
muchocine_dataset

DatasetDict({
    train: Dataset({
        features: ['review_body', 'review_summary', 'star_rating'],
        num_rows: 3872
    })
})